Authors: Dinu, Hofmarcher

Date: 17-03-2021

---

This file is part of the "Deep Reinforcement Learning" lecture material. The following copyright statement applies to all code within this file.

Copyright statement:
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

## Enable GPU Acceleration

---
Before you start exploring this notebook make sure that GPU support is enabled.
To enable the GPU backend for your notebook, go to **Edit** → **Notebook Settings** and set **Hardware accelerator** to **GPU**. 

---


# Install required packages and import modules

Install OpenAI Gym and dependencies to render the environments

In [ ]:
# Install dependencies
!apt update
!apt-get install -y xvfb x11-utils ffmpeg
!pip install gym==0.17.3 pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*
!pip install onnx onnx2pytorch
# Install environments
!pip install gym[box2d]

In [ ]:
%matplotlib inline

# ==============[DEBUGGING SEED]==============
# Don't forget to remove seeding and/or
# test on multiple seeds.
seed = None
import random
if seed: random.seed(seed)
import numpy as np
if seed: np.random.seed(seed)
import torch
if seed: torch.manual_seed(seed)

# Imports
import os
import time
import shutil
import copy
import zipfile
from time import sleep

# PyTorch imports
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.distributions import Categorical
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
from torchvision.transforms import Compose, ToTensor, Grayscale, ToPILImage
# Onnx model-export imports
import onnx
from onnx2pytorch import ConvertModel

# Auxiliary Python imports
import math
import glob
import io
import base64
from tqdm.notebook import tqdm
from time import sleep, time, strftime
from collections import namedtuple

# Environment import and set logger level to display error only
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only

# Plotting and notebook imports
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns; sns.set()

# Plotting and notebook imports
from IPython.display import HTML, clear_output
from IPython import display
from ipywidgets import Output

# start virtual display
from pyvirtualdisplay import Display
pydisplay = Display(visible=0, size=(640, 480))
pydisplay.start()

# Download Dataset and Unzip File

In contrast to Imitation Learning, data in Offline RL is not always perfect and can contain good and bad samples.

## Mixed Data

This data was collected by a PPO agent and logged during the training process. It contains expert as well as imperfect data samples.

In [ ]:
!wget --no-check-certificate 'https://cloud.ml.jku.at/s/CdYdidkkBpFgcED/download' -O train_mixed.zip

In [ ]:
# select as a data root the mixed demonstratoins directory
data_root = 'data_mixed'

In [ ]:
with zipfile.ZipFile('train_mixed.zip', 'r') as zip_ref:
    os.makedirs(data_root, exist_ok=True)
    zip_ref.extractall(data_root)

# Auxiliary Methods

In [ ]:
class Logger():
    def __init__(self, logdir, params=None):
        self.basepath = os.path.join(logdir, strftime("%Y-%m-%dT%H-%M-%S"))
        os.makedirs(self.basepath, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        if params is not None and os.path.exists(params):
            shutil.copyfile(params, os.path.join(self.basepath, "params.pkl"))
        self.log_dict = {}
        self.dump_idx = {}

    @property
    def param_file(self):
        return os.path.join(self.basepath, "params.pkl")

    @property
    def onnx_file(self):
        return os.path.join(self.basepath, "model.onnx")

    @property
    def log_dir(self):
        return os.path.join(self.basepath, "logs")

    def log(self, name, value):
        if name not in self.log_dict:
            self.log_dict[name] = []
            self.dump_idx[name] = -1
        self.log_dict[name].append((len(self.log_dict[name]), time(), value))
    
    def get_values(self, name):
        if name in self.log_dict:
            return [x[2] for x in self.log_dict[name]]
        return None
    
    def dump(self):
        for name, rows in self.log_dict.items():
            with open(os.path.join(self.log_dir, name + ".log"), "a") as f:
                for i, row in enumerate(rows):
                    if i > self.dump_idx[name]:
                        f.write(",".join([str(x) for x in row]) + "\n")
                        self.dump_idx[name] = i


def plot_metrics(logger):
    train_loss  = logger.get_values("training_loss")
    env_score  = logger.get_values("env_score")
    
    fig = plt.figure(figsize=(15,5))
    ax1 = fig.add_subplot(131, label="train")
    ax2 = fig.add_subplot(132, label="score")

    ax1.plot(train_loss, color="C0")
    ax1.set_ylabel("Loss", color="black")
    ax1.set_xlabel("Epoch", color="black")
    ax1.tick_params(axis='x', colors="black")
    ax1.tick_params(axis='y', colors="black")
    ax1.set_ylim((0, 10))

    ax2.plot(env_score, color="C1")
    ax2.set_ylabel("Score", color="black")
    ax2.set_xlabel("Epoch", color="black")
    ax2.tick_params(axis='x', colors="black")
    ax2.tick_params(axis='y', colors="black")
    ax2.set_ylim((-100, 1000))

    fig.tight_layout(pad=2.0)
    plt.show()


def print_action(action):
    print("Left %.1f" % action[0] if action[0] < 0 else "Right %.1f" % action[0] if action[0] > 0 else "Straight")
    print("Throttle %.1f" % action[1])
    print("Break %.1f" % action[2])


"""
Utility functions to enable video recording of gym environment and displaying it
"""
def concatenate_videos(video_dir):
    """
    Merge all mp4 videos in video_dir.
    """
    outfile = os.path.join(video_dir, 'merged_video.mp4')
    cmd = "ffmpeg -i \"concat:"
    mp4list = glob.glob(os.path.join(video_dir, '*.mp4'))
    tmpfiles = []
    # build ffmpeg command and create temp files
    for f in mp4list:
        file = os.path.join(video_dir, "temp" + str(mp4list.index(f) + 1) + ".ts")
        os.system("ffmpeg -i " + f + " -c copy -bsf:v h264_mp4toannexb -f mpegts " + file)
        tmpfiles.append(file)
    for f in tmpfiles:
        cmd += f
        if tmpfiles.index(f) != len(tmpfiles)-1:
            cmd += "|"
        else:
            cmd += f"\" -c copy  -bsf:a aac_adtstoasc {outfile}"
    # execute ffmpeg command to combine videos
    os.system(cmd)
    # cleanup
    for f in tmpfiles + mp4list:
        if f != outfile:
            os.remove(f)
    # --
    return outfile


def show_video(video_dir):
    """
    Show video in the output of a code cell.
    """
    # merge all videos
    mp4 = concatenate_videos(video_dir)    
    if mp4:
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")


# Convert RBG image to grayscale and normalize by data statistics
def rgb2gray(rgb, norm=True):
    # rgb image -> gray [0, 1]
    gray = np.dot(rgb[..., :], [0.299, 0.587, 0.114])
    if norm:
        # normalize
        gray = gray / 128. - 1.
    return gray


def hide_hud(img):
    img[84:] = 0
    return img


# Use to download colab parameter file
def download_colab_model(param_file):
    from google.colab import files
    files.download(param_file)


def save_as_onnx(torch_model, sample_input, model_path):
    torch.onnx.export(torch_model,             # model being run
                    sample_input,              # model input (or a tuple for multiple inputs)
                    f=model_path,              # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=13,          # the ONNX version to export the model to - see https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    )

# Dataloader

In [ ]:
# Action space (map from continuous actions for steering, throttle and break to 25 action combinations)
action_mapping = [
    (0, 0, 0),  # no action
    (0, 0.5, 0),  # half throttle
    (0, 1, 0),  # full trottle
    (0, 0, 0.5),  # half break
    (0, 0, 1),  # full break
    # steering left with throttle/break control
    (-0.5, 0, 0),  # half left
    (-1, 0, 0),  # full left
    (-0.5, 0.5, 0),  # half left
    (-1, 0.5, 0),  # full left
    (-0.5, 1, 0),  # half left
    (-1, 1, 0),  # full left
    (-0.5, 0, 0.5),  # half left
    (-1, 0, 0.5),  # full left
    (-0.5, 0, 1),  # half left
    (-1, 0, 1),  # full left
    # steering right with throttle/break control
    (0.5, 0, 0),  # half right
    (1, 0, 0),  # full right
    (0.5, 0.5, 0),  # half right
    (1, 0.5, 0),  # full right
    (0.5, 1, 0),  # half right
    (1, 1, 0),  # full right
    (0.5, 0, 0.5),  # half right
    (1, 0, 0.5),  # full right
    (0.5, 0, 1),  # half right
    (1, 0, 1)  # full right
]

# create transition object for partial demonstrations
Transition = namedtuple('Transition', ['states', 'actions', 'next_states', 'rewards', 'dones'])

# Since the demonstrations are partial files assuming that the collected data is too
# large to fit into memory at once the Demonstration class utilizes an object 
# from the ParialDataset class to load and unload files from the file system.
# This is a typical use case for very large datasets and should give you an idea 
# how to handle such issues.  
class Demonstration(object):
    def __init__(self, root_path):
        assert (os.path.exists(root_path))
        self.root_path = root_path
        # assign list of data files found in the data root directory
        self.data_files = sorted(os.listdir(root_path))

    def __len__(self):
        # this count returns the number of files in the data root folder
        return len(self.data_files)

    def load(self, idx):
        # select an index at random from all files
        file_name = self.data_files[idx]
        file_path = os.path.join(self.root_path, file_name)
        # load the selected file
        data = np.load(file_path)
        # get the respective properties from the files
        states = data["states"]
        actions = data["actions"]
        next_states = data["next_states"]
        rewards = data["rewards"]
        dones = data["dones"]
        # clean the memory from the data file
        del data
        # return the transitions
        return Transition(states=states, actions=actions, next_states=next_states, rewards=rewards, dones=dones)

# Itereates over the dataset subset in the known manner.
class PartialDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data.states)

    def __getitem__(self, idx):
        # select stack of states
        states = self.data.states[idx]
        # select followup action, next_state, reward and done flag
        action = self.data.actions[idx]
        next_state = self.data.next_states[idx]
        reward = self.data.rewards[idx]
        done = self.data.dones[idx]

        return states, action, next_state, reward, done

# Inspect data

In [ ]:
img_stack = 4
show_hud = True
batchsize = 128
epochs = 100
use_colab_autodownload = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

# BCQ Implementation

In [ ]:
# Defining a Q-Network for predicting and evaluating the next action given a state
class QNet(nn.Module):
    def __init__(self, img_stack, n_units_out):
        super(QNet, self).__init__()
        self.n_units_in = img_stack
        self.n_units_out = n_units_out
        # =========== YOUR CHANGES =============
        # ######################################
        # Use the network architecture of your choice
        # to extract the features and compute the
        # q values, and logits of the policy.
        # [HINT]: Don't use Softmax, LogSoftmax or similar
        # non parametric layers, since ONNX does not support this.
        # ######################################
        # 1) CNN feature extraction
        # 2) q values output head
        # 3) policy output head

    @staticmethod
    def _weights_init(m):
        if isinstance(m, nn.Conv2d):
            init.kaiming_uniform_(m.weight, a=math.sqrt(5))
            if self.bias is not None:
                fan_in, _ = init._calculate_fan_in_and_fan_out(m.weight)
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(m.bias, -bound, bound)

        elif instance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # =========== YOUR CHANGES =============
        # ######################################
        # Use the network architecture of your choice
        # to extract the features and compute the
        # q values, and logits of the policy
        # ######################################
        # 1) extract features with an CNN
        # 2) compute q values
        # 3) compute policy logits

        # returns q-function, log action probability and action logits
        return q_vals, pi_logits

In [ ]:
# Agent for the Discrete Batch-Constrained deep Q-Learning (BCQ) https://github.com/sfujim/BCQ/tree/master/discrete_BCQ
class ORLAgent(object):
    def __init__(
        self, 
        logger,
        img_stack, # image stack
        threshold=0.3, # threshold to bias away actions
        eval_eps=0.001, # action sampling epsilon
        # =========== YOUR CHANGES =============
        # ######################################
        # Find proper hyperparameters for 
        # discount factor, tau and adam optimizer
        # ######################################
        discount=..., # discount factor for Q-value computation
        lambda_=..., # regularization parameter
        tau=..., # parameter for exponential moving average of Q parameter updates
        optimizer="Adam", # optimizer 
        optimizer_parameters={ # hyperparameters for optimizer
			"lr": ...,
			"eps": ...
		}
	):
        self.logger = logger
        self.num_actions = len(action_mapping)

        # Create Q-network
        self.Q = QNet(img_stack, self.num_actions).to(device)
        # Create a copy of the current Q-Network as the target network
        self.Q_target = copy.deepcopy(self.Q)
        # Initialize optimizer
        self.Q_optimizer = getattr(torch.optim, optimizer)(self.Q.parameters(), **optimizer_parameters)
        # Q value discount factor
        self.discount = discount

        # Target update rule exponential move average hyperparameter
        self.tau = tau
        # Evaluation hyperparameter for action selection
        self.eval_eps = eval_eps
        # Threshold for biasing unlikely actions away
        self.threshold = threshold
        # Regularization hyperparameter
        self.lambda_ = lambda_

        num_trainable_params = sum(p.numel() for p in self.Q.parameters() if p.requires_grad)
        print("Trainable Parameters: {}".format(num_trainable_params))

    def _get_action_idx(self, state):
        # =========== YOUR CHANGES =============
        # ######################################
        # Get the action index based on the state provided
        # using the Q-Network
        # ######################################
        # 1) get current state q values and policy logits
        # 2) retriev probabilities through softmax
        # 3) create action mask using the threshold
        # 4) compute the next action index with the weighted q values and mask
        return action_idx

    def select_action(self, state):
        # Select action according to policy with probability (1-eps)
        # otherwise, select random action
        if np.random.uniform(0,1) > self.eval_eps:
            with torch.no_grad():
                next_action_idx = self._get_action_idx(state)
                return action_mapping[next_action_idx]
        else:
            # Randomly select an action uniformly
            next_action_idx = np.random.randint(self.num_actions)
            return action_mapping[next_action_idx]

    @torch.enable_grad()
    def train(self, state, action, next_state, reward, done):
        # Compute the target Q value
        with torch.no_grad():
            action_idx = self._get_action_idx(next_state)
            # Get target q-function
            q, _ = self.Q_target(next_state)
            # =========== YOUR CHANGES =============
            # ######################################
            # Calculate the target q values to 
            # update the Q-Network
            # ######################################
            # 1) compute the target Q-value
            target_Q = ...

        # Get current Q estimate
        current_Q, logits = self.Q(state)
        # Gather actions along dimension
        current_Q = current_Q.gather(1, action)
        # Get log probabilities from logits
        log_probs = self.log_softmax(logits)

        # =========== YOUR CHANGES =============
        # ######################################
        # Compute the loss based on the q values,
        # the policy constrain and an optional 
        # regularization.
        # ######################################
        # 1) compute Q loss using the smoothed L1 loss
        # 2) compute policy loss via the negative log-likelihood between log probabilites and demonstration actions
        # 3) regularize based on logits 
        # 4) compute total loss
        # 5) take a backward step on the Q function
        # 6) update target network by polyak by iterating over the Q-Network and target Q-Network parameters
        #    use tau parameter to compute the exponential moving average Q-Network parameters and update the target network

        # Return loss
        return loss.cpu().item()

    def mode(self, mode):
        # switch networks between evaluation and train mode
        if mode == 'train':
            self.Q.train()
        else:
            self.Q.eval()

    def save(self, param_file, sample):
        torch.save(self.Q.state_dict(), param_file)
        save_as_onnx(self.Q, sample, f'{param_file}_onnx')
        # download param file
        if use_colab_autodownload: download_colab_model(param_file)

    def load_param(self, param_file):
        self.Q.load_state_dict(torch.load(param_file, map_location="cpu"))    

# Define Training and Validation Routines

In [ ]:
def train_epoch(agent, train_set, logger, epoch, pbar):
    # Switch to train mode
    agent.mode('train')
    # Initialize helpers variables
    ts_len = len(train_set)
    running_loss = None
    alpha = 0.3
    # =========== [OPTIONAL] CHANGES =============
    # ############################################
    # [Hint]: Accessing the file system is slow and you can
    # reshape your data / load multiple files in to speed up 
    # training.
    # ############################################
    # Iterate over the list of demonstration files
    for i, idx in enumerate(BatchSampler(SubsetRandomSampler(range(ts_len)), 1, False)):
        # Load the selected index from the filesystem
        data = train_set.load(idx[0])
        # Create dataset from loaded data sub-set
        partial = PartialDataset(data)
        # Create dataloader
        loader = DataLoader(partial, batch_size=batchsize, num_workers=1, shuffle=True, drop_last=False, pin_memory=True)
        l_len = len(loader)
        # Iterate over parial dataset
        for j, (s, a, s_, r, d) in enumerate(loader):
            # Adjust types, shape and push to device
            s = s.float().to(device)
            a = a.long().unsqueeze(1).to(device)
            s_ = s_.float().to(device)
            r = r.float().unsqueeze(1).to(device)
            d = d.float().unsqueeze(1).to(device)
            # Train the respective agent
            loss = agent.train(s, a, s_, r, d)
            # Update running average loss
            running_loss = loss if running_loss is None else loss * alpha + (1 - alpha) * running_loss
            # Update info in the progress bar
            pbar.set_postfix_str("Epoch: %03d/%03d Partial: %03d/%03d Idx: %03d/%03d Loss: %.4f" % (epoch+1, epochs, i+1, ts_len, j+1, l_len, running_loss))
    return running_loss, s  # s serves as sample input for saving the model in ONNX format

# Evaluate the agent in the real environment

In [ ]:
class Env():
    """
    Environment wrapper for CarRacing 
    """
    def __init__(self, img_stack, show_hud=True, record_video=True, seed=None):
        self.record_video=record_video
        # Create gym environment
        self.gym_env = gym.make('CarRacing-v0')
        if seed:
            print(f"Environment seed: {seed}")
            self.gym_env.seed(seed)
            self.gym_env.action_space.seed(seed)
        self.env, self.video_dir = self.wrap_env(self.gym_env)
        self.action_space = self.env.action_space
        self.img_stack = img_stack
        self.show_hud = show_hud

    def reset(self, raw_state=False):
        self.env, self.video_dir = self.wrap_env(self.gym_env)
        self.rewards = []
        img_rgb = self.env.reset()
        img_gray = rgb2gray(img_rgb)
        if not self.show_hud:
            img_gray = hide_hud(img_gray)
        self.stack = [img_gray] * self.img_stack
        if raw_state:
            return np.array(self.stack), np.array(img_rgb)
        else:
            return np.array(self.stack)

    def step(self, action, raw_state=False):        
        # for i in range(self.img_stack):
        img_rgb, reward, done, _ = self.env.step(action)            
        # accumulate reward
        self.rewards.append(reward)            
        # if no reward recently, end the episode
        die = True if np.mean(self.rewards[-np.minimum(100, len(self.rewards)):]) <= -1 else False
        if done or die:
            self.env.close()
        img_gray = rgb2gray(img_rgb)
        if not self.show_hud:
            img_gray = hide_hud(img_gray)
        # add to frame stack  
        self.stack.pop(0)
        self.stack.append(img_gray)
        assert len(self.stack) == self.img_stack
        # --
        if raw_state:
            return np.array(self.stack), np.sum(self.rewards[-1]), done, die, img_rgb
        else:
            return np.array(self.stack), np.sum(self.rewards[-1]), done, die

    def render(self, *arg):
        return self.env.render(*arg)

    def close(self):
        self.env.close()
  
    def wrap_env(self, env):
        """
        Wrapper for recording video of the environment.
        """
        outdir = f"./videos/"
        if os.path.exists(outdir):
            shutil.rmtree(outdir)
        os.makedirs(outdir, exist_ok=True)
        if self.record_video:
            env = Monitor(env, outdir, force=True)
        return env, outdir


@torch.no_grad()
def run_episode(agent, n_runs=1, record_video=False, logger=None, pbar=None):
    agent.mode('eval')
    score_avg = None
    alpha = 0.3
    env_seeds = [np.random.randint(1e7) for _ in range(n_runs)]
    for i in range(n_runs):
        # Create new environment object
        env = Env(img_stack=img_stack, record_video=record_video, seed=env_seeds[i])
        state = env.reset()
        done_or_die = False
        score = 0
        while not done_or_die:
            t_state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            action = agent.select_action(t_state)
            state, reward, done, die = env.step(action)
            score += reward
            if pbar:
                pbar.set_postfix_str("Env Evaluation - Run {:03d} Score: {:.2f}".format(i+1, score))
            if done or die:
                done_or_die = True
            sleep(0.001)
        env.close()
        score_avg = score if score_avg is None else score * alpha + (1 - alpha) * score_avg
        print(f"Evaluation run {i} completed!")
    return score_avg

# Train your agent

In [ ]:
# Specify the google drive mount here if you want to store logs and weights there (and set it up earlier)
logger = Logger("logdir")
print("Saving state to {}".format(logger.basepath))

In [ ]:
# Load Dataset
train_set = Demonstration(data_root)

In [ ]:
# Create new agent
agent = ORLAgent(logger, img_stack=img_stack)

# Optionally load existing parameter file
#param_file = 'logdir/2021-03-18T16-44-29/params.pkl'
#agent.load_param(param_file)

In [ ]:
# Training
epoch_iter = range(epochs)
out = Output()
display.display(out)
with tqdm(epoch_iter) as pbar:
    for i_ep in pbar:
        print(f"Starting training epoch {i_ep+1}/{epochs}")
        # plot current training state
        if i_ep > 0:
            with out:
                display.clear_output(wait=True)
                plot_metrics(logger)
        # train
        train_loss, sample = train_epoch(agent, train_set, logger, i_ep, pbar)
        logger.log("training_loss", train_loss)
        # =========== [OPTIONAL] CHANGES =============
        # ############################################
        # Go full Offline RL if you feel up to it. :)
        # [Hint]: Evaluate in the environment - strictly speaking this is not allowed in pure Offline RL!!!
        # But we ease the task a bit and avoid that you are flying blind all the time.
        # Otherwise you would be only allowed to test once you submit to the challenge server.
        # If you are really looking for a challenge feel free to remove this line and make a train/eval data split from the demonstrations.
        # ############################################
        score = run_episode(agent, logger=logger, pbar=pbar)
        logger.log("env_score", score)
        # store logs
        logger.dump()
        # store weights
        print("Saving state to {}".format(logger.basepath))
        save_file_path = f'{logger.param_file}_%03d' % i_ep
        agent.save(save_file_path, sample)

clear_output(wait=True)
print("Saved state to {}".format(logger.basepath))
print("Trainable Parameters: {}".format(num_trainable_params))
print("[%03d] Training Loss: %.4f" % (i_ep + 1, train_loss))
plot_metrics(logger)

# Visualize Agent Interactions

### Put the agent into a real environment

Let's see how the agent is doing in the real environment

In [ ]:
# select agent you want to evaluate
agent = ORLAgent(logger, img_stack=img_stack)

# load parameter
#param_file = 'logdir/2021-03-29T09-24-10/params.pkl_009'
#agent.load_param(param_file)

# run episode with recording and show video
run_episode(agent, n_runs=1, record_video=True)
show_video(env.video_dir)